<a href="https://colab.research.google.com/github/jeehoshin/deep_learning/blob/master/pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##00. Introduction to PyTorch

https://pytorch.org/tutorials/beginner/basics/intro.html

### Tensors

In [ ]:
import torch
from __future__ import print_function

In [ ]:
#초기화되지 않은 행렬? 을 생성한다고 한다.
x = torch.empty(5,3)
print(x)

tensor([[-6.7107e+07,  3.0938e-41,  3.3631e-44],
        [ 0.0000e+00,         nan,  0.0000e+00],
        [ 1.1578e+27,  1.1362e+30,  7.1547e+22],
        [ 4.5828e+30,  1.2121e+04,  7.1846e+22],
        [ 9.2198e-39,  7.0374e+22,  5.0948e-14]])


In [ ]:
#초기화된 행렬 생성하기
x = torch.rand(5,3)
print(x)

tensor([[0.3971, 0.9358, 0.9143],
        [0.7568, 0.7411, 0.7003],
        [0.6285, 0.2647, 0.4358],
        [0.1992, 0.0544, 0.4419],
        [0.8574, 0.5432, 0.5056]])


In [ ]:
print(torch.zeros(5,3 ))

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [ ]:
#데이터를 바로 torch로 만들 수 있다.
x = torch.tensor([5.5,3])
print(x)

tensor([5.5000, 3.0000])


그냥 ```numpy```와 같은 방법이라고 생각하면 될 것 같다 

### 연산(Operations)

- inplace 방식
 : _를 붙이면 됨

In [ ]:
x = torch.tensor([5.5,3])
y = torch.tensor([4.4,2])

y.add_(x)
print(y)

tensor([9.9000, 5.0000])


- ```torch.view```

크기나 모양을 바꾸고 싶을 때 사용한다.

In [ ]:
x = torch.randn(4,4)
print(x)

tensor([[-0.0052, -0.8665,  0.9701, -1.5311],
        [ 0.3906, -0.7641,  0.1062, -1.3814],
        [ 0.0789, -1.6968, -0.2207,  1.5098],
        [-0.6114, -1.5097,  1.9319, -0.6095]])


In [ ]:
y = x.view(16)
z = x.view(-1,8) #우선 여덟개의 열로 먼저 잡고, 알아서 행을 채워 넣으라는 의미
y

tensor([-0.0052, -0.8665,  0.9701, -1.5311,  0.3906, -0.7641,  0.1062, -1.3814,
         0.0789, -1.6968, -0.2207,  1.5098, -0.6114, -1.5097,  1.9319, -0.6095])

In [ ]:
z

tensor([[-0.0052, -0.8665,  0.9701, -1.5311,  0.3906, -0.7641,  0.1062, -1.3814],
        [ 0.0789, -1.6968, -0.2207,  1.5098, -0.6114, -1.5097,  1.9319, -0.6095]])

### torch - numpy 변환

In [ ]:
import numpy as np

a_np = np.ones(5)
a_torch = torch.ones(5)

#torch to numpy
b_np = a_torch.numpy()
print(b_np)

#numpy to torch
b_torch = torch.from_numpy(a_np)
print(b_torch)

[1. 1. 1. 1. 1.]
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


### Datasets & Dataloaders

```DataLoader```로 편하게 batch와 shuffle을 지정할 수 있다고 한다.

### Transforms

우리의 훈련에 맞게 데이터를 변환시켜주는 로직임.

## 01. Build the Neural Network

```torch.nn```(Neural Network)를 통해서 모든 것들이 이뤄진다. nn.Module로 우리의 모델을 쌓는다고 생각하면 된다

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device)) #colab에서는 cuda를 사용할 수 있는 듯

Using cuda device


### 클래스 정의

nn.Module의 subclassing을 통해서 우리의 nn을 구축해야 한다.

모든 subclassed 모듈은 forward가 정의되어야 한다고 함.

In [ ]:
class NeuralNetwork(nn.Module) : 
  def __init__(self) : 
    super(NeuralNetwork, self).__init__() #NeuralNetwork를 가지고 온다.
    self.flatten = nn.Flatten() #이제 이 클래스를 생성할 때, flatten을 생성해야 한다. 
    self.linear_relu_stack = nn.Sequential(
                    nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
    )

  #.forward 메서드를 활용할 수 있는 것이다. 
  def forward(self, x) : 
    x = self.flatten(x) #아마 input을 의미할 것이다. 결국 x부터 순전파가 시작되어야 하므로
    logits = self.linear_relu_stack(x) #위에서 정의한 것을 사용
    return logits


class를 만들었으니 우리는 NeuralNetwork라는 인스턴스를 만들 수 있다. 이를 model이라는 객체에 보내본다.


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [ ]:
x = torch.rand(1,28,28, device = device)
#이렇게 데이터를 넣어주면 된다. model.forward()를 바로 call 해서는 안된다고 한다.
logits = model(x)
pred_probability = nn.Softmax(dim = 1)(logits)
print(pred_probability)
y_pred = pred_probability.argmax(1) #예측된 애들 중 가장 max값의 자리를 뽑아준다.
print(f"Predicted class: {y_pred}")

tensor([[0.1005, 0.0980, 0.0980, 0.0988, 0.0984, 0.1028, 0.1042, 0.0980, 0.1030,
         0.0984]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([6], device='cuda:0')


### 모델 레이어

layer에 대해서 살펴보는 section

In [ ]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


1) nn.Flatten

2D 28X28을 784의 픽셀로 변환시켜주는 층(신용평가 모형에서는 사용하지 않을 듯 싶다)

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


2) nn.Linear

input을 linear combination으로 만들어주는 층! tensorflow의 Dense라고 생각하면 되지 않을까

In [ ]:
layer1 = nn.Linear(in_features=28*28, #들어오는 피쳐의 모양
                   out_features= 20 )  #결과적으로 출력하는 데이터의 모양
hidden1 = layer1(flat_image)  #이렇게 객체를 구축하고 이후에 데이터를 넣어도 되는 듯
print(hidden1.size())
#확실히 out neuron이 20으로 줄어들었다.

torch.Size([3, 20])


3) nn.ReLU





linear 결합된 데이터를 활성화함수를 통해 다음 층으로 넘길 준비를 해야한다.

다음 결과값에서 볼 수 있듯, ReLU치면 input이 0보다 크면 그대로, 0보다 작으면 0으로 만들어준다

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4531,  0.2868,  0.6521, -0.0104, -0.1304,  0.3019, -0.1753, -0.7636,
         -0.1414,  0.0111,  0.2135, -0.0786, -0.3138,  0.2846, -0.3343,  0.6432,
          0.0766, -0.3644,  0.3334,  0.8441],
        [ 0.3398,  0.2675,  0.3450, -0.0525,  0.2513,  0.1388, -0.0738, -0.3668,
         -0.0509,  0.4824,  0.1312, -0.5371, -0.0148,  0.3241, -0.4197,  0.7596,
          0.4527, -0.1869,  0.4113,  0.7505],
        [ 0.6044,  0.6043,  0.5447, -0.2666,  0.3034, -0.2122, -0.2580, -0.1551,
          0.3552,  0.2295,  0.2579, -0.3911, -0.3979,  0.0754, -0.3484,  0.3364,
          0.2296, -0.2769,  0.1237,  0.6627]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.4531, 0.2868, 0.6521, 0.0000, 0.0000, 0.3019, 0.0000, 0.0000, 0.0000,
         0.0111, 0.2135, 0.0000, 0.0000, 0.2846, 0.0000, 0.6432, 0.0766, 0.0000,
         0.3334, 0.8441],
        [0.3398, 0.2675, 0.3450, 0.0000, 0.2513, 0.1388, 0.0000, 0.0000, 0.0000,
         0.4824, 0.1312, 0.0000, 0.0000, 0.3241, 0.000

4) nn.sequential

우리가 쌓아온 층들을 모아두는 컨테이너 박스라고 생각하면 된다. 대신 순차적으로 쌓아두어야 함. 

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10) #들어온 20개의 값들을 10개로 바꿔준다.
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

5) nn.Softmax

마지막 활성화 함수로서 출력함수라고도 한다

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

6) Model Parameters

아주 편하게도 nn.Module은 자동적으로 우리의 모델을 보여주고 파라미터도 보여준다고 한다.

In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0079,  0.0234,  0.0034,  ...,  0.0070, -0.0234, -0.0277],
        [ 0.0063,  0.0215, -0.0172,  ...,  0.0092,  0.0245, -0.0258]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0056, 0.0070], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0337, -0.0351,  0.0026,  ...,  0.0019, -0.0069, -0.0337],
        [-0.0389,  0.0321,  0.0350,  ...,  0.0270,  0.0308, -0.0312]],
       device='cuda:0

## 02. Autograd

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

역전파 알고리즘을 수월하게 수행해주는 function

built-in 된 자동 미분 함수인데, 미분할 수 있는 그래프에서는 모두 자동으로 계산해 줌

In [11]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b #Matrix product of two tensors.
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

<img src="https://pytorch.org/tutorials/_images/comp-graph.png">

출처 : https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

In [12]:
print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f1886f130d0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f1886f13110>


### Computing Gradients

우리는 loss를 가중치로 미분을 하고, loss를 편향으로 미분을 해야하는, 두번의 미분이 필요하게 된다.

loss.backward()를 해주면, 각각의 gradient를 구할 수 있다.

이 경우 w.grad와 b.grad로 gradient를 구할 수 있다.

> 주의할 것은 같은 graph에서 backward를 여러번 할 수 없다는 것. 하고 싶다면 retain_graph = True를 해줘야 한다.(근데 해보면 값이 달라진다)

> 이렇게 값이 달라지는 이유는, Pytorch는 ```backward``` 에서 gradient를 계속 누적하기 때문이라고 함.

> 따라서 알맞는 gradient를 구하기 위해서는, gradient를 다시 0으로 맞춰줘야 한다고 함. 실제 상황에서는 *Optimizer*가 우리를 도와줄 것이라고 하는데...

> 

In [24]:
#x의 경우 requires_grad = True로 해두지 않았기 때문에 grad를 뽑지 못한다.
loss.backward(retain_graph=True)
print(w.grad)
print(b.grad)

tensor([[3.8056, 1.7441, 3.8606],
        [3.8056, 1.7441, 3.8606],
        [3.8056, 1.7441, 3.8606],
        [3.8056, 1.7441, 3.8606],
        [3.8056, 1.7441, 3.8606]])
tensor([3.8056, 1.7441, 3.8606])


### Disabling Gradient Tracking

모형을 학습시킨 뒤, input을 단순히 forwarding만 하고 싶을 경우가 있을 것이다. 그때 ```torch.no_grad()```를 사용함

1. 몇몇 파라미터를 고정하고 싶을 때

2. 단순히 포워딩만 사용할 때 계산시간을 단축시킬 수 있음

In [25]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad(): #이거로 감싸주는 것이다.
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


```detach```메서드를 사용하는 것도 한 방법

In [26]:
z = torch.matmul(x, w)+b
z_det = z.detach() #이렇게 떼어준 객채를 만든다. 아마 이 방법이 더 편할지도?
print(z_det.requires_grad)

False


## 03. Optimization

In [27]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Hyperparameters

- epoch의 수
- batch size : 한 epoch에 들어갈 데이터 
- learning rate


In [28]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

학습 epoch는 두 과정으로 진행된다고 함.

- The Train Loop : 

- The validation Loop : 

### Loss function

학습을 진행하면서 최소화 하고자 하는 함수. 

우리의 모형을 태워 만든 예측값과 실제값의 차이를 최대한 줄이려고 함

In [30]:
loss_fn = nn.CrossEntropyLoss()

### Optimizer(최적화기)

In [ ]:
optimizer = torch.optim.SGD(model.parateters(), lr = learning_rate)
# 여기서는 lbfgs도 된다. 
# torch.optim.LBFGS()

- ```optimizer.zero_grad()```를 사용해서 위에서 생긴 역전파의 문제를 해결해야 한다. 이걸 안 해주면 계속해서 더해지게 된다. (가중치가 누적되는 Pytorch의 특성은, 추후 RNN에서 도움이 된다고 한다.)

- ```loss.backwards()```로 역전파를 진행해준다.

- 기울기를 구했다면, ```optimizer.step()```을 통해 구해진 기울기로 parameter들을 조정해준다.(아마 여기서 파라미터는 가중치를 말하는 것일듯)

### Implementation

In [38]:
def train_loop(dataloader, mode, loss_fn, optimizer) : 
  size = len(dataloader.dataset)
  for batch, (x,y) in enumerate(dataloader) : 
    #forward를 진행
    pred = model(x)

    #지정해준 loss_fn으로 계산
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad() #가중치 초기화
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [39]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.305079  [    0/60000]
loss: 2.296262  [ 6400/60000]
loss: 2.292182  [12800/60000]
loss: 2.279119  [19200/60000]
loss: 2.272808  [25600/60000]
loss: 2.264206  [32000/60000]
loss: 2.257181  [38400/60000]
loss: 2.252377  [44800/60000]
loss: 2.232034  [51200/60000]
loss: 2.212020  [57600/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 0.034889 

Epoch 2
-------------------------------
loss: 2.243945  [    0/60000]
loss: 2.242600  [ 6400/60000]
loss: 2.226997  [12800/60000]
loss: 2.200779  [19200/60000]
loss: 2.189287  [25600/60000]
loss: 2.180068  [32000/60000]
loss: 2.164477  [38400/60000]
loss: 2.161250  [44800/60000]
loss: 2.131160  [51200/60000]
loss: 2.083040  [57600/60000]
Test Error: 
 Accuracy: 49.5%, Avg loss: 0.033144 

Epoch 3
-------------------------------
loss: 2.167981  [    0/60000]
loss: 2.165261  [ 6400/60000]
loss: 2.138304  [12800/60000]
loss: 2.077301  [19200/60000]
loss: 2.061318  [25600/60000]
loss: 2.066042  [32000/600

## 04. Save and Load the Model

In [42]:
import torch
import torch.onnx as onnx
import torchvision.models as models

### saving and loading model weights

In [41]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [43]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval() #이것을 꼭 진행해줘야 한다고 함함

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

saving and loading model with shapes

In [44]:
torch.save(model, 'model.pth')

In [45]:
model = torch.load('model.pth')